In [9]:

import sys
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from models import resnet_model
from models import efficient_model
from models import mobilenet_model

data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
sys.path.append(data_path)

# 4. Importer DataLoaders depuis dataloaders.py
from DataLoaders import DataLoader

import tensorflow as tf
import tensorflow.keras.backend as K
import gc

In [10]:
print(f"Keras version: {tf.keras.__version__}")


Keras version: 3.8.0


In [11]:

data_loader = DataLoader(path="C:/Users/nessa/Downloads/sipakmed_new6/")
datasets = data_loader.get_datasets()

train_with_aug = datasets["train_with_augmentation"]
test_set = datasets["test"]

Found 551 images belonging to 3 classes.
Found 551 images belonging to 3 classes.
Found 177 images belonging to 3 classes.


In [12]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1),
    EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
]

In [15]:
def train_and_evaluate(model_class, model_name):

    print("\n==============================")
    print(f"  Entraînement du modèle : {model_name}")
    print("==============================\n")

    model_obj = model_class(
        input_shape=(224, 224, 3),
        num_classes=3,
        learning_rate=1e-4
    )
    model = model_obj.get_model()

    # ---- PHASE 1 ----
    model.fit(
        train_with_aug,
        epochs=15,
        validation_data=test_set,
        callbacks=callbacks,
        verbose=2
    )

    # ---- FINE TUNING ----
    for layer in model.layers:
        layer.trainable = True

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_with_aug,
        epochs=10,
        validation_data=test_set,
        callbacks=callbacks,
        verbose=2
    )

    # ================ ÉVALUATION ================
    print(f"\nÉvaluation du modèle {model_name} :")
    test_loss, test_accuracy = model.evaluate(test_set, verbose=2)
    print(f"> Accuracy : {test_accuracy * 100:.2f}%")

    # ==== EXTRACTION LABELS ====
    y_true = []
    for i in range(len(test_set)):
        _, label = test_set[i]   # (image, one-hot)
        y_true.append(label[0])

    y_true = np.array(y_true)
    y_true_classes = np.argmax(y_true, axis=1)

    # ==== PRÉDICTIONS ====
    y_pred = model.predict(test_set)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # ==== AUC ====
    auc = roc_auc_score(y_true, y_pred, multi_class='ovr')
    print(f"> AUC : {auc:.4f}")

    # ==== RAPPORT ====
    print("\nRapport de classification :")
    print(classification_report(
        y_true_classes,
        y_pred_classes,
        target_names=data_loader.class_names
    ))

    # ==== MATRICE ====
    print("\nMatrice de confusion :")
    print(confusion_matrix(y_true_classes, y_pred_classes))

    return {
        "model": model_name,
        "accuracy": test_accuracy,
        "auc": auc
    }


In [21]:

# Trouver les classes dans les modules
ResNet50_Model = resnet_model.ResNet50_Model  # ou le nom exact trouvé
EfficientNetB0_Model = efficient_model.EfficientNetB0_Model
MobileNetV2_Model = mobilenet_model.MobileNetV2_Model
results = []

results.append(train_and_evaluate(ResNet50_Model, "ResNet50"))
K.clear_session()
gc.collect()

results.append(train_and_evaluate(EfficientNetB0_Model, "EfficientNetB0"))
K.clear_session()
gc.collect()

results.append(train_and_evaluate(MobileNetV2_Model, "MobileNetV2"))
K.clear_session()
gc.collect()



  Entraînement du modèle : ResNet50



c:\Users\nessa\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
18/18 - 365s - 20s/step - accuracy: 0.6951 - loss: 5.2253 - val_accuracy: 0.5876 - val_loss: 5.5913 - learning_rate: 1.0000e-04
Epoch 2/15
18/18 - 238s - 13s/step - accuracy: 0.9093 - loss: 4.6921 - val_accuracy: 0.7571 - val_loss: 5.0673 - learning_rate: 1.0000e-04
Epoch 3/15
18/18 - 237s - 13s/step - accuracy: 0.9238 - loss: 4.5140 - val_accuracy: 0.7627 - val_loss: 5.0350 - learning_rate: 1.0000e-04
Epoch 4/15
18/18 - 257s - 14s/step - accuracy: 0.9437 - loss: 4.3439 - val_accuracy: 0.8136 - val_loss: 4.8688 - learning_rate: 1.0000e-04
Epoch 5/15
18/18 - 229s - 13s/step - accuracy: 0.9564 - loss: 4.2015 - val_accuracy: 0.7797 - val_loss: 5.1226 - learning_rate: 1.0000e-04
Epoch 6/15
18/18 - 227s - 13s/step - accuracy: 0.9637 - loss: 4.0654 - val_accuracy: 0.9096 - val_loss: 4.3708 - learning_rate: 1.0000e-04
Epoch 7/15
18/18 - 225s - 12s/step - accuracy: 0.9837 - loss: 3.9118 - val_accuracy: 0.9040 - val_loss: 4.3077 - learning_rate: 1.0000e-04
Epoch 8/15
18/18 - 222s - 1

c:\Users\nessa\OneDrive\Bureau\tp\Apprentisaage-1\src\models\mobilenet_model.py:17: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = MobileNetV2(


Epoch 1/15
18/18 - 155s - 9s/step - accuracy: 0.4828 - loss: 5.2869 - val_accuracy: 0.3446 - val_loss: 5.4448 - learning_rate: 1.0000e-04
Epoch 2/15
18/18 - 90s - 5s/step - accuracy: 0.7568 - loss: 4.7565 - val_accuracy: 0.2599 - val_loss: 5.9485 - learning_rate: 1.0000e-04
Epoch 3/15
18/18 - 94s - 5s/step - accuracy: 0.8367 - loss: 4.5418 - val_accuracy: 0.2429 - val_loss: 6.2402 - learning_rate: 1.0000e-04
Epoch 4/15
18/18 - 89s - 5s/step - accuracy: 0.8693 - loss: 4.3837 - val_accuracy: 0.2373 - val_loss: 5.9471 - learning_rate: 1.0000e-04
Epoch 5/15
18/18 - 93s - 5s/step - accuracy: 0.8820 - loss: 4.2621 - val_accuracy: 0.2373 - val_loss: 6.0999 - learning_rate: 1.0000e-04
Epoch 6/15
18/18 - 93s - 5s/step - accuracy: 0.9165 - loss: 4.1133 - val_accuracy: 0.2316 - val_loss: 6.7058 - learning_rate: 1.0000e-04
Epoch 7/15
18/18 - 87s - 5s/step - accuracy: 0.9274 - loss: 4.0027 - val_accuracy: 0.2373 - val_loss: 6.7848 - learning_rate: 1.0000e-04
Epoch 8/15
18/18 - 93s - 5s/step - accur

0

In [22]:
print("\n\n==============================")
print("     TABLEAU FINAL DES SCORES")
print("==============================\n")

for r in results:
    print(f"{r['model']}: Accuracy = {r['accuracy']:.4f} | AUC = {r['auc']:.4f}")




     TABLEAU FINAL DES SCORES

ResNet50: Accuracy = 0.9492 | AUC = 0.9960
EfficientNetB0: Accuracy = 0.9379 | AUC = 0.9934
MobileNetV2: Accuracy = 0.3107 | AUC = 0.5869
